In [1]:
import time
from tqdm import tqdm
import numpy as np
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.core.utils import ChromeType
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromiumService

**Getting each season URL's**

In [2]:
if os.path.isdir('data'):
    shutil.rmtree('data')
    os.makedirs('data/summaries')

In [3]:
BASE_URL = 'https://breakingbad.fandom.com' 

r = requests.get(BASE_URL + '/wiki/Category:Seasons_(Breaking_Bad)')

soup = bs(r.content, 'lxml')

In [4]:
all_items_S = soup.find_all('div', class_ = 'category-page__members-wrapper')[-1]
season_list = all_items_S.find_all('li', class_ = 'category-page__member')

season_link = []

for season in season_list:
    season_link.append((season.find('a').get('title'), BASE_URL + season.find('a').get('href')))

season_link

[('Season 1 (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki/Season_1_(Breaking_Bad)'),
 ('Season 2 (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki/Season_2_(Breaking_Bad)'),
 ('Season 3 (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki/Season_3_(Breaking_Bad)'),
 ('Season 4 (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki/Season_4_(Breaking_Bad)'),
 ('Season 5A (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki/Season_5A_(Breaking_Bad)'),
 ('Season 5B (Breaking Bad)',
  'https://breakingbad.fandom.com/wiki/Season_5B_(Breaking_Bad)')]

**Episode Summary**

In [5]:
# https://pypi.org/project/webdriver-manager/

def call_webdriver():
    
    return webdriver.Chrome(service = ChromiumService(ChromeDriverManager(chrome_type = ChromeType.CHROMIUM).install()))

In [6]:
def get_summmary(episode_link, season_name):
    
    driver = call_webdriver()
    driver.minimize_window()
    driver.get(episode_link)
    time.sleep(5)
    all_para = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/h3[1]//following::p')
    
    with open(f'data/summaries/{season_name}.txt', 'a') as file:
        for para in all_para:
            if para.aria_role =='paragraph':
                file.write(para.text + '\n')
                
    # list_para = [para.text for para in all_para if para.aria_role =='paragraph']
    driver.close()
    
    # return list_para

In [7]:
def get_starring(episode_link):
    
    r = requests.get(episode_link)
    soup = bs(r.content, 'lxml')
    
    starring_block = soup.find('div', class_ = 'tabber wds-tabber')
    charactres_class = starring_block.find_all('div', class_ = 'wds-tab__content')

    # for char in charactres_class:
    #     for ch in char.select('li'):
    #         print(ch.text)
        
    starrings = [ch.text for char in charactres_class for ch in char.select('li')]
        
    # starrings = [star.text for star in soup.select('div.tabber.wds-tabber table li')]
    
    # for char in starrings:
    #     print(char.text)
    
    return starrings

In [8]:
episode_links, characters = {}, {}

for season, link in tqdm(season_link):
    
    r = requests.get(link)

    soup = bs(r.content, 'lxml')
    episodes = ['https://breakingbad.fandom.com' + x.find('a').get('href') for x in soup.select('#gallery-0 div.thumb')]
    
    season = season.split(' (')[0].replace(' ', '_')
    episode_links[season] = episodes
    
    characts = []
    
    for episode in episodes:
        get_summmary(episode, season)
        characts.append(get_starring(episode))
        
    characters[season] = list(np.concatenate(characts))
    
    
    # cnt += len(episodes)
    break
        
print(f'Number of episodes recoreded   :: {np.concatenate(list(episode_links.values())).shape[0]}')

  0%|                                                                          | 0/6 [07:07<?, ?it/s]

Number of episodes recoreded   :: 7


In [9]:
with open('data/season_nd_episode_links.txt', 'w') as file:
        file.write(str(episode_links))

In [ ]:
Season_1 = ['Season_1'] * len(characters['Season_1'])
Season_2 = ['Season_2'] * len(characters['Season_2'])
Season_3 = ['Season_3'] * len(characters['Season_3'])
Season_4 = ['Season_4'] * len(characters['Season_4'])
Season_5A = ['Season_5A'] * len(characters['Season_5A'])
Season_5B = ['Season_5B'] * len(characters['Season_5B'])

seas = Season_1 + Season_2 + Season_3 + Season_4 + Season_5A + Season_5B

chars = characters['Season_1'] + characters['Season_2'] + characters['Season_3'] + characters['Season_4'] + characters['Season_5A'] + characters['Season_5B']

In [23]:
character_df = pd.DataFrame({'Season' : seas, 'Characters' : chars})
character_df.to_csv('data/character_df.csv', index = False)
character_df.head()

,Season_1,Season_2
0,Bryan Cranston as Walter White,Bryan Cranston as Walter White
1,Anna Gunn as Skyler White,Anna Gunn as Skyler White
2,Aaron Paul as Jesse Pinkman,Aaron Paul as Jesse Pinkman
3,Dean Norris as Hank Schrader,Dean Norris as Hank Schrader
4,Betsy Brandt as Marie Schrader,Betsy Brandt as Marie Schrader
...,...,...
128,Matthew Page as Chemical Plant Guard,Matthew Page as Chemical Plant Guard
129,Jacob Mullikan as Prospective Buyer,Jacob Mullikan as Prospective Buyer
130,Charles Dowdy as Mr. Wilson,Charles Dowdy as Mr. Wilson
131,Cliff Gravel as Teacher,Cliff Gravel as Teacher
